# TSP Nearest Insertion Heuristic

So we have a set of 8 cities or vertex $V = \{1,2,3,4,5,6,7,8\}$ with a corresponding $D = d_{i,j} i, j \in V$ distance matrix between every city

In [1]:
V = collect(1:8)

8-element Vector{Int64}:
 1
 2
 3
 4
 5
 6
 7
 8

In [2]:
D = 
[0 89 87 38 33 71 59 54
89 0 32 59 65 39 45 61
87 32 0 50 75 17 64 79
38 59 50 0 40 33 50 56
33 65 75 40 0 62 26 21
71 39 17 33 62 0 57 70
59 45 64 50 26 57 0 16
54 61 79 56 21 70 16 0]

8×8 Matrix{Int64}:
  0  89  87  38  33  71  59  54
 89   0  32  59  65  39  45  61
 87  32   0  50  75  17  64  79
 38  59  50   0  40  33  50  56
 33  65  75  40   0  62  26  21
 71  39  17  33  62   0  57  70
 59  45  64  50  26  57   0  16
 54  61  79  56  21  70  16   0

And we will have a $T$ set which will represent the ordered tour of our TSP

In [3]:
T = Int64[]

Int64[]

And all of this minimizing the distance between cities, while connecting every single one of them once.

To start, let's pick the smallest tour available, by getting the minimal distance in the matrix. Now, because I am lazy I will use the `findmin` function defined in Julia but it will return the zero elements of the main diagonal. So what I will do first is to replace all of those with undesirable distances. A naive approach would be to simply do a `replace` but it will mess with other 0s in the matrix, so let´s stick to the diagonal. I could also use the LinearAlgebra stdlib but I wanna try stuff by myself

In [4]:
for i ∈ 1:8
    D[i, i] = 999999
end
D

8×8 Matrix{Int64}:
 999999      89      87      38      33      71      59      54
     89  999999      32      59      65      39      45      61
     87      32  999999      50      75      17      64      79
     38      59      50  999999      40      33      50      56
     33      65      75      40  999999      62      26      21
     71      39      17      33      62  999999      57      70
     59      45      64      50      26      57  999999      16
     54      61      79      56      21      70      16  999999

In [5]:
mindis, coords = findmin(D)
x = coords[1]
y = coords[2]
println("Min distance $mindis found between $x and $y cities")

Min distance 16 found between 8 and 7 cities


So we add those to our $T$ tour

In [6]:
push!(T, x)
push!(T, y)

2-element Vector{Int64}:
 8
 7

And we create a $\bar{V}$ without the cities we've already selected. I will create a helper function for that

In [7]:
function remove!(V, item)
    deleteat!(V, findall(x->x==item, V))
end
V̄ = copy(V)
remove!(V̄, x)
remove!(V̄, y)

6-element Vector{Int64}:
 1
 2
 3
 4
 5
 6

So while $\bar{V}$ is not empty we shall iterate, adding the nearest node to our current tour and selecting the best place for insertion. We can use many metrics to define "nearest node", either the mean or the sum of distances or simply the minimum value. In this case, I will be using the sum of all distances of node $i$ to $T$

In [8]:
distances = Int64[]
for i in V̄
    dist = 0
    for j in T
        dist = dist + D[i,j]
    end
    push!(distances, dist)
end
distances

6-element Vector{Int64}:
 113
 106
 143
 106
  47
 127

Now that we know all of our distances, let´s pick the smallest one available and get the node that is represented by such distance to $T$

In [9]:
mindis, î = findmin(distances)
println("Min distance to T $mindis found from city $î")

Min distance to T 47 found from city 5


We must determine the best place to insert this node. I am too bored to type the LaTeX in of the equation.

In [10]:
insertion = Tuple[]
for q in 1:length(T)
    if q ≠ length(T)
        distances = D[T[q], î] + D[î,T[q+1]] - D[T[q], T[q+1]]
        insert = tuple(T[q], distances)
        push!(insertion, insert)
    else
        distances = D[T[q], î] + D[î,T[q-1]] - D[T[q], T[q-1]]
        insert = tuple(T[q], distances)
        push!(insertion, insert)
    end
end
sums = [x[2] for x in insertion]
mindis, idx = findmin(sums)
city = insertion[idx][1]
println("Inserting after city $city with a weight of $mindis")

Inserting after city 8 with a weight of 31


In this case it doesn't matter really where do we insert the new node 5, so lets just insert it at 8.

In [11]:
push!(T, î)
remove!(V̄, î)
println("Current tour T: $T")

Current tour T: [8, 7, 5]


So lets create our loop iterating $\bar{V}$

In [12]:
#while !isempty(V̄)
    distances = Int64[]
    for i in V̄
        dist = 0
        for j in T
            dist = dist + D[i,j]
        end
        push!(distances, dist)
    end
    println(distances)
#end

[146, 171, 218, 146, 189]
